# Modified SVM-mentalBERT Simulation

## Text Preparation

In [ ]:
# add required libraries
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
!pip install -q nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [ ]:
# set random seed to reproduce the same result every time
np.random.seed(500)

In [ ]:
# import the corpus
Corpus = pd.read_csv("500_Reddit_users_posts_labels.csv", encoding='latin-1')

## Text Vector Representation

## MentalBERT for data imbalance

## MentalBERT for dimensionality reduction

## Preprocessed data into SVM

## Evaluate

## Hyperparameter Tuning if necessary